In [3]:
!pip install torch

     |████████████████████████████████| 150.6 MB 168 kB/s  eta 0:00:01     |█████████████████████████████▌  | 138.9 MB 30.8 MB/s eta 0:00:01


In [4]:
!pip3 install torch_geometric

In [8]:
!pip install torch torchvision

     |████████████████████████████████| 1.7 MB 3.1 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
import torch
from torch_geometric.data import Data
import torch.nn.functional as F

# num_classes = 3
# edge_index = torch.tensor([[0, 1, 1, 2, 3, 2],
#                            [1, 0, 2, 1, 2, 3]], dtype=torch.long)
# x = torch.tensor([[], [], [], []], dtype=torch.float)
# y = torch.tensor([[0, 1, 0], [0, 0, 1], [1, 0, 1], [1, 0, 0]], dtype=torch.float)
# train_mask = torch.tensor([True, True, False, False], dtype=torch.bool)
# test_mask = torch.tensor([False, False, True, True], dtype=torch.bool)
# x
# data = Data(x=x, edge_index=edge_index, y=y, train_mask=train_mask, test_mask=test_mask)
# data

In [188]:
from torch.utils.data import random_split

data = torch.load('GNN_data_class_weights.pt')

# Assuming 'dataset' is your entire dataset
dataset_size = data.x.shape[0]
train_size = int(0.8 * dataset_size)
test_size = dataset_size - train_size

train_dataset, test_dataset = random_split(data.x, [train_size, test_size])

# Now you can create masks based on the lengths of train_dataset and test_dataset
train_mask = torch.zeros(dataset_size, dtype=torch.bool)
test_mask = torch.zeros(dataset_size, dtype=torch.bool)
train_mask[list(train_dataset.indices)] = True
test_mask[list(test_dataset.indices)] = True

data.test_mask = test_mask
data.train_mask = train_mask
num_classes = data.y.shape[1]
data

Data(x=[2432, 384], edge_index=[2, 3319], edge_attr=[3319], y=[2432, 71], y_weights=[71], articles=[2432], categories=[71], test_mask=[2432], train_mask=[2432])

In [189]:
node1 = data.edge_index[0][9]
node2 = data.edge_index[1][9]
print(data.articles[node1])
print(data.articles[node2])
print(data.edge_attr[9])

Pasta al forno
Al forno
tensor(1.6167)


In [15]:
# tensorboard
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

AttributeError: module 'numpy' has no attribute 'typeDict'

In [190]:
import torch_geometric.nn

class GCNModel(torch.nn.Module):
    def __init__(self, num_nodes, hidden_size=128):
        super(GCNModel, self).__init__()
        self.node_embedding = torch.nn.Embedding(num_nodes, hidden_size)
        # Initialize the embeddings with small random values
        torch.nn.init.normal_(self.node_embedding.weight, std=0.1)
        # self.conv1 = torch_geometric.nn.GCNConv(hidden_size, hidden_size)
        self.conv1 = torch_geometric.nn.GCNConv(hidden_size, 16)
        self.conv2 = torch_geometric.nn.GCNConv(16, num_classes)

    def forward(self, data):
        edge_index = data.edge_index
        edge_attr = data.edge_attr
        # x = self.node_embedding.weight
        x = data.x

        # x = self.conv1(x, edge_index)
        x = self.conv1(x, edge_index, edge_attr)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index, edge_attr)
        return F.softmax(x, dim=1)
    
class GCNModelNodeFeat(torch.nn.Module):
    def __init__(self, num_nodes, num_features, hidden_size=128):
        super(GCNModelNodeFeat, self).__init__()
        # self.conv1 = torch_geometric.nn.GCNConv(hidden_size, hidden_size)
        self.conv1 = torch_geometric.nn.GCNConv(num_features, hidden_size)
        self.conv2 = torch_geometric.nn.GCNConv(hidden_size, hidden_size)
        self.conv3 = torch_geometric.nn.GCNConv(hidden_size, hidden_size)
        self.conv4 = torch_geometric.nn.GCNConv(hidden_size, num_classes)

    def forward(self, data):
        edge_index = data.edge_index
        edge_attr = data.edge_attr
        # x = self.node_embedding.weight
        x = data.x

        # x = self.conv1(x, edge_index)
        x = self.conv1(x, edge_index, edge_attr)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index, edge_attr)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv3(x, edge_index, edge_attr)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv4(x, edge_index, edge_attr)
        return F.softmax(x, dim=1)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = GCNModel(len(data.x)).to(device)
print(data.x.shape[1])
model = GCNModelNodeFeat(len(data.x), data.x.shape[1]).to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

# sets the model in training mode
# When you call model.train(), PyTorch enables features such as dropout and 
# batch normalization, which are typically used during training but not during inference
model.train()
for epoch in range(1000):
    optimizer.zero_grad()
    out = model(data)
    loss = F.cross_entropy(out[data.train_mask], data.y[data.train_mask], weight=data.y_weights)
    loss.backward()
    if(epoch % 10 == 0):
        print(epoch, loss)
    # writer.add_scalar("Loss/train", loss, epoch)
    optimizer.step()

# writer.flush()

384
0 tensor(6.5904, grad_fn=<DivBackward1>)
10 tensor(6.5894, grad_fn=<DivBackward1>)
20 tensor(6.5818, grad_fn=<DivBackward1>)
30 tensor(6.5574, grad_fn=<DivBackward1>)
40 tensor(6.5488, grad_fn=<DivBackward1>)
50 tensor(6.5196, grad_fn=<DivBackward1>)
60 tensor(6.4936, grad_fn=<DivBackward1>)
70 tensor(6.4803, grad_fn=<DivBackward1>)
80 tensor(6.4590, grad_fn=<DivBackward1>)
90 tensor(6.4347, grad_fn=<DivBackward1>)
100 tensor(6.4111, grad_fn=<DivBackward1>)
110 tensor(6.4130, grad_fn=<DivBackward1>)
120 tensor(6.3760, grad_fn=<DivBackward1>)
130 tensor(6.3600, grad_fn=<DivBackward1>)
140 tensor(6.3492, grad_fn=<DivBackward1>)
150 tensor(6.3497, grad_fn=<DivBackward1>)
160 tensor(6.3276, grad_fn=<DivBackward1>)
170 tensor(6.3213, grad_fn=<DivBackward1>)
180 tensor(6.3183, grad_fn=<DivBackward1>)
190 tensor(6.3032, grad_fn=<DivBackward1>)
200 tensor(6.2976, grad_fn=<DivBackward1>)
210 tensor(6.2970, grad_fn=<DivBackward1>)
220 tensor(6.2931, grad_fn=<DivBackward1>)
230 tensor(6.2824,

In [186]:
from sklearn.metrics import f1_score, precision_score, recall_score

model.eval()
pred = model(data)
# Convert probabilities to binary predictions
binary_pred = (pred >= 1.0/num_classes).float()

for i in range(20):
    print(data.articles[i])
    boolean_mask = binary_pred[i] == 1.0
    filtered_categories = [category for category, mask in zip(data.categories, boolean_mask) if mask]
    print(filtered_categories)


# Check if each prediction matches the label for each instance
correct = (binary_pred[data.test_mask] == data.y[data.test_mask]).sum() / num_classes
acc = int(correct) / int(data.test_mask.sum())

precision = precision_score(data.y[data.test_mask], binary_pred[data.test_mask], average='micro')
print(f'Precision: {precision:.4f}')
recall = recall_score(data.y[data.test_mask], binary_pred[data.test_mask], average='micro')
print(f'Recall: {recall:.4f}')
f1 = f1_score(data.y[data.test_mask], binary_pred[data.test_mask], average='micro')
print(f'F1 Score: {f1:.4f}')

Al forno
['Deep fried foods', 'Bread stubs', 'Steamed foods']
Bacon Explosion
['Deep fried foods']
Beef Wellington
['Deep fried foods', 'Savoury pies', 'Steamed foods']
Papas arrugadas
['Deep fried foods']
Chicken Kiev
['Deep fried foods', 'Savoury pies', 'Steamed foods']
Chicken tikka
['Deep fried foods', 'Savoury pies', 'Steamed foods']
Clambake
['Deep fried foods']
Cordon bleu (dish)
['Deep fried foods', 'Steamed foods']
Frikkadel
['Sweet breads', 'Deep fried foods', 'Savoury pies', 'Bread stubs', 'Steamed foods']
German baked apples
['Deep fried foods', 'Bread stubs']
Gratin
['Deep fried foods', 'Savoury pies', 'Steamed foods']
Kolach (bread)
['Sweet breads']
Kugelis
['Deep fried foods']
Meatloaf
['Deep fried foods', 'Bread stubs']
Baked milk
['Sweet breads', 'Bread stubs', 'Steamed foods']
Pasta al forno
['Deep fried foods', 'Bread stubs', 'Steamed foods']
Pickert
['Deep fried foods']
Pom (dish)
['Deep fried foods', 'Steamed foods']
Rumbledethumps
['Deep fried foods']
Sajji
['Deep

In [ ]:
import networkx as nx
import torch_geometric.utils
from matplotlib import colormaps
import matplotlib.pyplot as plt
from numpy import random

g = torch_geometric.utils.to_networkx(data, to_undirected=True)

node_list = dict(map(lambda i,j : (i,j) , g.nodes, data.articles))
font_size = 6

degrees = dict(g.degree).values()
node_size = [(d+1) * 50 for d in degrees]

idx = 0
for n, l in node_list.items():
    node_list[n] = l if node_size[idx] > 400 else (l if node_size[idx] > 200 and random.rand() > 0.95 else (l if random.rand() > 0.85 else ''))
    idx += 1

pred_list = [p.tolist() for p in pred]
node_pred = [float(p.index(max(p))) for p in pred_list]
node_color = [colormaps['tab20'](p/num_classes) for p in node_pred]

fig = plt.figure(1, figsize=(12, 10), dpi=100)
nx.draw(g, labels=node_list, pos=nx.spring_layout(g, k=0.2, iterations=50), font_size=font_size, node_size=node_size, node_color=node_color)